In [78]:
from keras import Input, Model, regularizers
from keras.datasets import mnist
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
%matplotlib inline

In [145]:
# This is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats
input_shape = 28**2
initializer = tf.keras.initializers.GlorotUniform()

# This is our input image
input_img = Input(shape=(input_shape,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu',activity_regularizer=regularizers.l1(10e-5),
                kernel_initializer=initializer)(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(input_shape, activation='sigmoid', kernel_initializer=initializer)(encoded)

# This model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

In [146]:
# This model maps an input to its encoded representation
encoder = Model(input_img, encoded)

In [147]:
# This is our encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# Create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [148]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [149]:
# Load MNIST dataset
(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 256.
x_test = x_test.astype('float32') / 256.
x_train = x_train.reshape(x_train.shape[0], 28**2)
x_test = x_test.reshape(x_test.shape[0], 28**2)
print(x_train.shape)
print(x_test.shape)

(60000, 784)
(10000, 784)


In [150]:
autoencoder.summary()

Model: "functional_67"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        [(None, 784)]             0         
_________________________________________________________________
dense_24 (Dense)             (None, 32)                25120     
_________________________________________________________________
dense_25 (Dense)             (None, 784)               25872     
Total params: 50,992
Trainable params: 50,992
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Train autoencoder
history = autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Epoch 1/50
235/235 [==============================] - 5s 21ms/step - loss: 0.2886 - val_loss: 0.2009
Epoch 2/50
235/235 [==============================] - 4s 18ms/step - loss: 0.1809 - val_loss: 0.1625
Epoch 3/50
235/235 [==============================] - 5s 20ms/step - loss: 0.1537 - val_loss: 0.1436
Epoch 4/50
235/235 [==============================] - 4s 18ms/step - loss: 0.1392 - val_loss: 0.1322
Epoch 5/50
235/235 [==============================] - 5s 20ms/step - loss: 0.1289 - val_loss: 0.1233
Epoch 6/50
235/235 [==============================] - 4s 17ms/step - loss: 0.1219 - val_loss: 0.1179
Epoch 7/50
235/235 [==============================] - 4s 17ms/step - loss: 0.1173 - val_loss: 0.11420s - loss: 0.
Epoch 8/50
235/235 [==============================] - 4s 17ms/step - loss: 0.1139 - val_loss: 0.1113
Epoch 9/50
195/235 [=======================>......] - ETA: 0s - loss: 0.1117

In [ ]:
# Encode and decode some digits
# Note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

In [ ]:
n = 10  # How many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

In [ ]:
#check output of each hidden layer
encoded_imgs = np.eye((encoding_dim))*1000
decoded_imgs = decoder.predict(encoded_imgs)


In [ ]:
n = encoding_dim  # How many digits we will display
fig = plt.figure(figsize=(20, 8))
for i in range(n):
    # Display original

    # Display reconstruction
    ax = plt.subplot(4,encoding_dim//4, 1+i)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.set_title(i)
fig.subplots_adjust(hspace=0.3)

In [ ]:
# summarize history for loss
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')